# DataMadness Group \#44 - F1 Analysis

In [37]:
import pandas as pd
import numpy as np
import requests
import zipfile
import io
import os

from matplotlib import pyplot as plt

## Loading Data

In [38]:
# Define the URL for the zip file and the local filename to use
url = 'http://ergast.com/downloads/f1db_csv.zip'
filename = 'f1db_csv.zip'

# Create a data directory if it doesn't exist
if not os.path.exists('../data'):
    os.makedirs('../data')

# Download the zip file
response = requests.get(url)

# Unzip the file into the data directory
with zipfile.ZipFile(io.BytesIO(response.content)) as zip_file:
    zip_file.extractall('../data')

dataframes = {}

# Loop through the csv files in the data directory
for csv_file in os.listdir('../data'):
    if csv_file.endswith('.csv'):
        # Read the csv file into a dataframe 
        df_name = csv_file.replace('.csv', '')
        dataframes[df_name] = pd.read_csv(f'../data/{csv_file}')

In [39]:
# races
races = dataframes['races']
print(races.shape)
races.head()

(1102, 18)


,raceId,year,round,circuitId,name,date,time,url,fp1_date,fp1_time,fp2_date,fp2_time,fp3_date,fp3_time,quali_date,quali_time,sprint_date,sprint_time
0,1,2009,1,1,Australian Grand Prix,2009-03-29,06:00:00,http://en.wikipedia.org/wiki/2009_Australian_G...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
1,2,2009,2,2,Malaysian Grand Prix,2009-04-05,09:00:00,http://en.wikipedia.org/wiki/2009_Malaysian_Gr...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
2,3,2009,3,17,Chinese Grand Prix,2009-04-19,07:00:00,http://en.wikipedia.org/wiki/2009_Chinese_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
3,4,2009,4,3,Bahrain Grand Prix,2009-04-26,12:00:00,http://en.wikipedia.org/wiki/2009_Bahrain_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
4,5,2009,5,4,Spanish Grand Prix,2009-05-10,12:00:00,http://en.wikipedia.org/wiki/2009_Spanish_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N


In [40]:
# results
results = dataframes['results']
print(results.shape)
results.head()

(25840, 18)


,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
0,1,18,1,1,22,1,1,1,1,10.0,58,1:34:50.616,5690616,39,2,1:27.452,218.300,1
1,2,18,2,2,3,5,2,2,2,8.0,58,+5.478,5696094,41,3,1:27.739,217.586,1
2,3,18,3,3,7,7,3,3,3,6.0,58,+8.163,5698779,41,5,1:28.090,216.719,1
3,4,18,4,4,5,11,4,4,4,5.0,58,+17.181,5707797,58,7,1:28.603,215.464,1
4,5,18,5,1,23,3,5,5,5,4.0,58,+18.014,5708630,43,1,1:27.418,218.385,1


In [41]:
# circuits
circuits = dataframes['circuits']
print(circuits.shape)
circuits.head()

(77, 9)


,circuitId,circuitRef,name,location,country,lat,lng,alt,url
0,1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.84970,144.96800,10,http://en.wikipedia.org/wiki/Melbourne_Grand_P...
1,2,sepang,Sepang International Circuit,Kuala Lumpur,Malaysia,2.76083,101.73800,18,http://en.wikipedia.org/wiki/Sepang_Internatio...
2,3,bahrain,Bahrain International Circuit,Sakhir,Bahrain,26.03250,50.51060,7,http://en.wikipedia.org/wiki/Bahrain_Internati...
3,4,catalunya,Circuit de Barcelona-Catalunya,Montmeló,Spain,41.57000,2.26111,109,http://en.wikipedia.org/wiki/Circuit_de_Barcel...
4,5,istanbul,Istanbul Park,Istanbul,Turkey,40.95170,29.40500,130,http://en.wikipedia.org/wiki/Istanbul_Park


In [42]:
# status
status = dataframes['status']
print(status.shape)
status.head()

(139, 2)


,statusId,status
0,1,Finished
1,2,Disqualified
2,3,Accident
3,4,Collision
4,5,Engine


In [43]:
# TODO keep doing the same, when adding a new table

## Question 1
### Does experience (years in Formula 1) correlate to the frequency of accidents (caused by driver)?

## Question 2

### At which tracks does starting position have a higher impact on the final position?

In [ ]:
# Testing merge

## Question 3

### How does the altitude of a racing circuit affect the number of mechanical failures?


In [44]:
# merge results and status to gain 'status'
df = results[['resultId', 'raceId', 'driverId', 'statusId']].merge(status, on='statusId')
df.head()

,resultId,raceId,driverId,statusId,status
0,1,18,1,1,Finished
1,2,18,2,1,Finished
2,3,18,3,1,Finished
3,4,18,4,1,Finished
4,5,18,5,1,Finished


In [45]:
# merge results newly obtained df and races to gain 'circuitId'
df = df.merge(races[['raceId', 'circuitId']], on='raceId')
df.head()

,resultId,raceId,driverId,statusId,status,circuitId
0,1,18,1,1,Finished,1
1,2,18,2,1,Finished,1
2,3,18,3,1,Finished,1
3,4,18,4,1,Finished,1
4,5,18,5,1,Finished,1


In [46]:
# merge newly obtained df and circuits to gain 'location', 'country' and 'alt'
df = df.merge(circuits[['circuitId', 'location', 'country', 'alt']], on='circuitId')
df['alt'] = pd.to_numeric(df['alt'], errors='coerce')
df.head()

,resultId,raceId,driverId,statusId,status,circuitId,location,country,alt
0,1,18,1,1,Finished,1,Melbourne,Australia,10.0
1,2,18,2,1,Finished,1,Melbourne,Australia,10.0
2,3,18,3,1,Finished,1,Melbourne,Australia,10.0
3,4,18,4,1,Finished,1,Melbourne,Australia,10.0
4,5,18,5,1,Finished,1,Melbourne,Australia,10.0


We noticed that there are altitudes missing for two races (Miami 2022 and Qatar 2022). Since both locations made their first appearance in 2022, we do not have altitude data from previous years and thus decided to drop these records.

In [47]:
df[df['alt'].isnull()]

,resultId,raceId,driverId,statusId,status,circuitId,location,country,alt
25760,25346,1051,1,1,Finished,78,Al Daayen,Qatar,NaN
25761,25347,1051,830,1,Finished,78,Al Daayen,Qatar,NaN
25762,25348,1051,4,1,Finished,78,Al Daayen,Qatar,NaN
25763,25349,1051,815,1,Finished,78,Al Daayen,Qatar,NaN
25764,25350,1051,839,1,Finished,78,Al Daayen,Qatar,NaN
25765,25351,1051,840,1,Finished,78,Al Daayen,Qatar,NaN
25766,25352,1051,832,1,Finished,78,Al Daayen,Qatar,NaN
25767,25353,1051,844,1,Finished,78,Al Daayen,Qatar,NaN
25768,25354,1051,846,11,+1 Lap,78,Al Daayen,Qatar,NaN
25769,25355,1051,20,11,+1 Lap,78,Al Daayen,Qatar,NaN


In [48]:
# drop records where we dont have 'alt' data
df.dropna(inplace=True)
df.head()

,resultId,raceId,driverId,statusId,status,circuitId,location,country,alt
0,1,18,1,1,Finished,1,Melbourne,Australia,10.0
1,2,18,2,1,Finished,1,Melbourne,Australia,10.0
2,3,18,3,1,Finished,1,Melbourne,Australia,10.0
3,4,18,4,1,Finished,1,Melbourne,Australia,10.0
4,5,18,5,1,Finished,1,Melbourne,Australia,10.0
